In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
import SimpleITK as sitk
import pydicom
from pydicom.tag import Tag
import sys
import csv
import pydicom as pdcm
import pandas as pd

In [2]:
pathtoPxList = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/PETasMarch27_Full.csv"

df = pd.read_csv(pathtoPxList)
df_new = df.copy()
df_new

,ID,CT,PET,Date,Name
0,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL
1,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E
2,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E
3,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL
4,17420,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,9/4/2017,PET WB Earl
...,...,...,...,...,...
759,9898346,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,6/28/2017,PET WB Uncorrected
760,9915436,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,2/6/2019,PET WB_EARL
761,9920297,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/4/2015,PET WB EARL
762,9935146,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,11/19/2019,Vision PET Earl


In [3]:
class DicomHeader:
    def __init__(self, ds):
        self.seriesDesc = ds.SeriesDescription
        try:
            self.corrImg = ds.CorrectedImage
        except: self.corrImg = None
        try: 
            self.petUnits = ds.Units
        except: self.petUnits = None
        try:
            self.seriesType = ds.SeriesType
        except: self.seriesType = None
        try:
            self.countsSource = ds.CountsSource
        except: self.countsSource = None
        try:
            self.randCorrMethod = ds.RandomsCorrectionMethod
        except: self.randCorrMethod = None
        try:
            self.attCorrMeth = ds.AttenuationCorrectionMethod
        except: self.attCorrMeth = None
        try:
            self.decayCorr = ds.DecayCorrection
        except: self.decayCorr = None
        try: 
            self.reccMethod = ds.ReconstructionMethod
        except: self.reccMethod = None
        try:
            self.scaterCorrMeth = ds.ScatterCorrectionMethod
        except: self.scaterCorrMeth =None
        self.imgType = ds.ImageType
        self.institution = ds.InstitutionName
        try:
            self.codeMeaning1 = ds.RelatedSeriesSequence[0][0x0040,0xa170][0].CodeMeaning
        except: self.codeMeaning1 = None    
        try:
            self.codeMeaning2 = ds.RelatedSeriesSequence[0][0x0040,0xa170][1].CodeMeaning
        except: self.codeMeaning2 = None
        

In [5]:
#read PET Corrected 
for count in range(len(df_new)):  
    pet_path = df_new['PET'][count]
    listofDCMs = os.listdir(pet_path)
    for singledcm in listofDCMs:
        ds = pydicom.dcmread(os.path.join(pet_path,singledcm),stop_before_pixels=True)
        if ds.Modality =='PT':
            header_class = DicomHeader(ds)
            break

    for attr in vars(header_class):
        df_new.loc[count,attr] = str(getattr(header_class, attr))
    break

In [6]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 192
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Positron Emission Tomography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.1005.30000015032408394935900052244
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.8.691.0.50
(0002, 0013) Implementation Version Name         SH: '1.2.0'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0016) SOP Class UID                       UI: Positron Emission Tomography Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.1005.30000015032408394935900052244
(0008, 0020) Study Date                          DA: '20150324'
(0008

In [4]:
new_columns = ["seriesDesc","corrImg","petUnits","seriesType","countsSource","randCorrMethod","attCorrMeth","decayCorr","reccMethod","scaterCorrMeth","imgType","institution","codeMeaning1","codeMeaning2"]
for col_name in new_columns:
    df_new = df_new.assign(**{col_name: 0})
df_new

,ID,CT,PET,Date,Name,seriesDesc,corrImg,petUnits,seriesType,countsSource,randCorrMethod,attCorrMeth,decayCorr,reccMethod,scaterCorrMeth,imgType,institution,codeMeaning1,codeMeaning2
0,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,5/7/2015,PET_E,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/24/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,17420,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,9/4/2017,PET WB Earl,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,9898346,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,6/28/2017,PET WB Uncorrected,0,0,0,0,0,0,0,0,0,0,0,0,0,0
760,9915436,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,2/6/2019,PET WB_EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
761,9920297,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,3/4/2015,PET WB EARL,0,0,0,0,0,0,0,0,0,0,0,0,0,0
762,9935146,//zkh/appdata/RTDicom/Projectline_modelling_lu...,//zkh/appdata/RTDicom/Projectline_modelling_lu...,11/19/2019,Vision PET Earl,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
saveCSV = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/PETasMarch27_FullW_withDCMHEader.csv"
df_new.to_csv(saveCSV, index=False)